In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd '/content/gdrive/My Drive/sem8/prml/prj'
!ls
current_loc=!pwd
print(current_loc)
from platform import python_version
print(python_version())

In [ ]:
#module imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import sklearn.cluster
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
!pip3 install catboost
import catboost
from catboost import CatBoostRegressor

In [ ]:
#data loads

train = pd.read_csv('data/train.csv')
songs = pd.read_csv('data/songs.csv')
song_labels = pd.read_csv('data/song_labels.csv')
test = pd.read_csv('data/test.csv')
save_for_later = pd.read_csv('data/save_for_later.csv')
dummy_submission = pd.read_csv('data/dummy_submission.csv')

In [ ]:
#rough0


#print(train[train['customer_id']=='O29219'])
#print(song_labels[song_labels['label_id']==30574])
#print(song_labels[song_labels['label_id']==8717]) #11557 11743

#print(train.iloc[[8101]])
#p=songs[songs['song_id']==5581]
#print(p['count0'],p['count1'],p['count2'])
#print(songs.iloc[[629]]['count0'],songs.iloc[[629]]['count1'],songs.iloc[[629]]['count2'])
#0.0  4614.0  5117.0

print(songs['released_year'].max())
print(songs['language'].unique())
#plt.plot(songs[songs['released_year']>=1860]]['released_year'])
#pd.qcut(songs['released_year'], q=4)
#language_mapping = {0: 'nan',1: 'eng',2:'en-US',3:'en-GB',4:'en-CA',}

#mapping
#Out: {1: 'A', 2: 'B', 3: 'C'}

#sample_df['variable'].map(mapping)
y = pd.get_dummies(songs['language'])
#print('pp: ',songs[songs['language'].isnull()])
print(songs['language'])
print(y)
#print('xi',(y.iloc[[3]]==0).all())

In [ ]:
#preprocessing songs

max_labels=10
song_labels_new = song_labels.groupby('label_id').sum().sort_values('count', ascending = False)
song_labels_new.reset_index(inplace = True)
principal_song_labels = song_labels_new['label_id'].to_numpy()[0:max_labels]  #highest labelled data is wantedly excluded!

for i in range(len(principal_song_labels)):
    label = song_labels[song_labels['label_id'] == principal_song_labels[i]]
    label = label.reset_index().drop(['index', 'label_id'], axis = 1)
    label[f'count{i}'] = label['count']
    label.drop(['count'], axis = 1, inplace = True)
    songs = pd.merge(songs, label, on = 'platform_id', how = 'left')
    songs[f'count{i}'] = songs[f'count{i}'].fillna(0)
    
songs = songs.drop(['platform_id'], axis = 1)
song_scores = train.merge(train.groupby('song_id').mean(), on = 'song_id', how = 'left')[['song_id', 'score_y']].drop_duplicates('song_id', keep = 'first')
songs = songs.merge(song_scores, on = 'song_id', how = 'left')
song_num_ratings = train['song_id'].value_counts().to_frame()
song_num_ratings['num_ratings'] = song_num_ratings['song_id']
song_num_ratings['song_id'] = song_num_ratings.index
songs = songs.merge(song_num_ratings, on = 'song_id', how = 'left')
songs.drop_duplicates('song_id', keep = 'first', inplace = True)

In [ ]:
#rough1

print(principal_song_labels)
print(song_labels)
print(songs)
print(train)

In [ ]:
#train preprocessing

f = pd.merge(train, save_for_later, on=['customer_id','song_id'], how='left', indicator='Exist')
train = f

X_train = pd.merge(train, songs, on = ['song_id'], how = 'left')
Y_train = X_train['score']
X_train.drop(['score'], axis = 1, inplace = True)
X_train.drop('song_id', axis = 1, inplace = True)
X_train['released_year']=abs(X_train['released_year'])
X_train['released_year'] = X_train['released_year'].fillna(-999)
X_train['language'] = X_train['language'].fillna('none')
X_train['number_of_comments'] = X_train['number_of_comments'].fillna(-999)
for i in range(max_labels):
    X_train[f'count{i}'] = X_train[f'count{i}'].fillna(0)
X_train['score_y']=X_train['score_y'].fillna(0)
X_train['num_ratings']=X_train['num_ratings'].fillna(0)
for i in range(max_labels):
    X_train[f'count{i}'] = X_train[f'count{i}'].fillna(0)


In [ ]:
#rough2


X_train.loc[(X_train.released_year < 1860), 'released_year'] = -999
print(X_train['released_year'])
print('hi: ',np.where(np.array(X_train['released_year'])<1860))
#X_train[:,X_train['released_year']<1860]['released_year']=-999
#plt.plot(X_train['released_year'])
#plt.show()
#X_train[X_train['released_year'].min()==X_train['released_year']]
#print(NaN in X_train['number_of_comments'].unique())
print(principal_song_labels)
print(X_train.columns)
print(X_train['num_ratings'].isnull().values.any())
print(X_train['Exist'])
print('hell0: ',np.array(pd.qcut(X_train[X_train['customer_id']=='O29219']['released_year'], q=4).value_counts()))
#X_train[1462:1464]
#X_train['released_year'].min()<0
#print(X_train['released_year'].min())
#print(X_train['released_year'].max())


In [ ]:
#feature vector compression
epsilon=1e-9
def softmax(z):  #computes softmax-values for each set of z along axis=-1
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis]
    return e_x / div

def squared_norm(z):  #computes squared-norm for each set of z along axis=-1
    assert len(z.shape) == 2
    return np.array(z)/(1e-9+np.linalg.norm(z,axis=-1))[:,np.newaxis]

def norm(z):  #computes norm for each set of z along axis=-1
    assert len(z.shape) == 2
    return np.array(z)/(1e-9+np.sum(z,axis=-1))[:,np.newaxis]

customers=X_train['customer_id'].unique()
check,max_customers=0,len(customers)
lang_one_hot=pd.get_dummies(X_train['language'])
lang_one_hot['customer_id']=X_train['customer_id']
compressed_features={}
count_cols=[]
for i in range(max_labels):
    count_cols.append('count'+str(i))
if check:
    print('Xtrain: ',X_train)
    print('lang_one_hot: ',lang_one_hot)
    print('count_cols: ',count_cols)
for customer in customers[0:max_customers]:
    c=np.where(customers==customer)[0][0]
    print('Processing ',customer,' : ',c,'/',len(customers))
    customer_data=X_train[X_train['customer_id']==customer]
    customer_scores=np.array(train[train['customer_id']==customer]['score'])
    label_counts=squared_norm(np.array(customer_data[count_cols]))
    year_cut=pd.cut(customer_data['released_year'], bins=[-1000, 1860, 1910, 1950, 2000, 2020])
    year_counts=squared_norm(np.array([year_cut.value_counts()[1:]]))
    lang_oh=np.array(lang_one_hot[lang_one_hot['customer_id']==customer].drop('customer_id', axis=1))
    lang_counts=squared_norm(np.array([np.sum(lang_oh*(customer_scores[:,np.newaxis]),axis=0)]))
    maxima_index=np.argmax(label_counts,axis=-1)
    soft_labels=squared_norm(np.array([np.sum(label_counts*(customer_scores[:,np.newaxis]**2),axis=0)]))
    if check:# or customer=='G23431':
        print('label_counts: ',label_counts)
        print('year_counts: ',year_counts)
        print('score_data: ',customer_scores)
        print('maxima_index: ',maxima_index)
        print('year_cut: ',year_cut)
        print('lang_one_hot: ',lang_oh)
        print('lang_counts: ',lang_counts)
        print('soft_labels: ',soft_labels)
    compressed_features[customer]=np.concatenate((soft_labels[0],year_counts[0],lang_counts[0]))
        
print('compressed_features: ',compressed_features)
#cust1=X_train[X_train['customer_id']=='O29219']

In [ ]:
#rough 3


import numpy as np
from sklearn.decomposition import PCA
fm=np.array(list(compressed_features.values()))[0:10]
print(fm.shape)
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(fm)
#pca.fit(fm)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print(principalComponents)

In [ ]:
#find best max_cluster to cluster datasets using hierarchical clustering
from sklearn.metrics.pairwise import cosine_similarity

import sklearn.cluster
feature_matrix=np.array(list(compressed_features.values()))
#feature_matrix_=cosine_similarity(feature_matrix)
print(feature_matrix[0])
#feature_matrix_=feature_matrix-feature_matrix.mean(axis=0)
from sklearn.cluster import KMeans
feature_matrix=np.array(list(compressed_features.values()))
#feature_matrix_=cosine_similarity(feature_matrix)
metrics1=[]
metrics2=[]
#metrics3=[]
for i in range(2,30):
    print('i: ',i)
    clustering=sklearn.cluster.AgglomerativeClustering(n_clusters=i,affinity='cosine',linkage='average').fit(squared_norm(feature_matrix))
    metrics1.append(metrics.silhouette_score(feature_matrix,clustering.labels_, metric='euclidean'))
    metrics2.append(metrics.calinski_harabasz_score(feature_matrix,clustering.labels_))
#    metrics3.append(kmeans.inertia_)
print(clustering.labels_)
#clustering=sklearn.cluster.AgglomerativeClustering(n_clusters=5).fit(feature_matrix)
print(clustering.labels_.shape)
#df=pd.DataFrame.from_dict(compressed_features,orient='index').transpose()
#df.to_csv('data/names.csv')
print(max(metrics1))
print(metrics1)
plt.plot(metrics1)
plt.plot(metrics2)
plt.show()

In [ ]:
#find best max_cluster to cluster datasets using kmeans clustering

from sklearn.cluster import KMeans
feature_matrix=np.array(list(compressed_features.values()))
#feature_matrix_=cosine_similarity(feature_matrix)
metrics1=[]
metrics2=[]
metrics3=[]
for i in range(2,30):
    print('i: ',i)
    kmeans = KMeans(n_clusters=i, random_state=0).fit(feature_matrix)
    metrics1.append(metrics.silhouette_score(feature_matrix,kmeans.labels_, metric='euclidean'))
    metrics2.append(metrics.calinski_harabasz_score(feature_matrix,kmeans.labels_))
    metrics3.append(kmeans.inertia_)
print(kmeans.labels_)
print(max(metrics1))
print(metrics1)
plt.plot(metrics1)
plt.show()

In [ ]:
#grouping customers into cohorts

max_clusters=2  #specify the max_cluster here after analyzing plots from above two cells (finding optimal cluster for hiearchical and k-means clustering)
feature_matrix=np.array(list(compressed_features.values()))
kmeans = KMeans(n_clusters=max_clusters, random_state=0).fit(feature_matrix)
metrics.silhouette_score(feature_matrix,kmeans.labels_, metric='euclidean')
metrics.calinski_harabasz_score(feature_matrix,kmeans.labels_)

groups={}
for label_id in range(len(kmeans.labels_)):
    label=kmeans.labels_[label_id]
    if label not in groups.keys():
        groups[label]=[]
    groups[label].append(customers[label_id])
print('groups: ',groups)
print('group keys: ',groups.keys())
for i in range(max_clusters):
    print('Length of cluster ',i,' : ',len(groups[i]))


In [ ]:
#rough4 --to plot dendorgrams (tough to visualize due to large data)

from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)



# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(feature_matrix)
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
#rough5

#for i in range(10):
#    plt.plot(X_trainee[X_trainee['customer_id']=='O29219']['count'+str(i)])
#    plt.plot(X_trainee[X_trainee['customer_id']=='I50343']['count'+str(i)])
#    plt.show()
print(compressed_features['O29219'][0:10])
print(compressed_features['I50343'][0:10])
print(compressed_features['H45532'][0:10])
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.cluster
#feature_matrix=np.array(list(compressed_features.values()))
cos1=cosine_similarity([compressed_features['O29219'][0:10]],[compressed_features['I50343'][0:10]])
cos2=cosine_similarity([compressed_features['H45532'][0:10]],[compressed_features['I50343'][0:10]])
cos3=cosine_similarity([compressed_features['O29219'][0:10]],[compressed_features['H45532'][0:10]])
print(cos1,cos2,cos3)
#print(X_trainee[X_trainee['customer_id']=='O29219']['count0'])
#print(train[train['customer_id']=='O29219']['score'])
#    print(X_trainee[X_trainee['customer_id']=='I50343']['count'+str(i)])

In [ ]:
#rough 6 --highly overfitting sklearn.tree!

from sklearn import tree
clf = tree.DecisionTreeRegressor()
#X_trainee=X_train[X_train.customer_id.isin(groups[0])]
#X_trainee.drop('customer_id', axis = 1, inplace = True)
#Y_trainee=train[train.customer_id.isin(groups[0])]['score']
X_trainee=X_train
Y_trainee=Y_train
print(X_trainee['language'])

X_trainee['customer_id']=X_trainee['customer_id'].astype('category')
X_trainee['language']=X_trainee['language'].astype('category')
cat_columns = X_trainee.select_dtypes(['category']).columns
print(cat_columns)
X_trainee[cat_columns] = X_trainee[cat_columns].apply(lambda x: x.cat.codes)
print(X_trainee['language'])

clf = clf.fit(X_trainee, Y_trainee)
#clf.predict([[1, 1]])
print(clf.predict(X_train))
print(Y_train)
loss = ((Y_train-clf.predict(X_train))**2).mean()

In [ ]:
class RmseMetric(object):  #metric used by catboostregressor (give as input argument to CatBoostRegressor)
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((approx[i] - target[i])**2)

        return error_sum, weight_sum

In [ ]:
#training pre-processing with multiple models for different groups
models=[]
for i in range(max_clusters):
    print('Training model #',i)
    X_trainee=X_train[X_train.customer_id.isin(groups[i])]
#    X_trainee.drop('customer_id', axis = 1, inplace = True)
    Y_trainee=train[train.customer_id.isin(groups[i])]['score']
    x_train, x_test, y_train, y_test = train_test_split(X_trainee, Y_trainee, train_size = 0.8)
    model = CatBoostRegressor()#eval_metric=RmseMetric())
    eval_dataset = catboost.Pool(data=x_test, label=y_test, cat_features=[0, 1, 3])
    model.fit(x_train, y_train, eval_set = eval_dataset, cat_features = [0, 1,3], use_best_model = True, plot = True)
    models.append(model)

In [ ]:
#rough 7 --'sample weights=x' in model.fit()

print(len(groups.keys()),groups[0])
x=X_train.customer_id.isin(groups[0])
print(x)
x = x*1
x=x+1
print(x)

In [ ]:
#training pre-processing with one single model for the whole dataset
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, train_size = 0.8)
model = CatBoostRegressor(learning_rate=0.5,eval_metric=RmseMetric())
#xxx=x_train.customer_id.isin(groups[0])
#xxx = xxx*1
#xxx=xxx+1
eval_dataset = catboost.Pool(data=x_test, label=y_test, cat_features=[0, 1,3])  #[0, 1, 3])
X_train.customer_id.isin(groups[i])
model.fit(x_train, y_train, eval_set = eval_dataset, cat_features = [0, 1,3],use_best_model = True, plot = True)

In [ ]:
#test pre-processing

#test = pd.merge(test, save_for_later, on=['customer_id','song_id'], how='left', indicator='Exist')
X_test = pd.merge(test, songs, on = ['song_id'], how = 'left')
X_test.drop('song_id', axis = 1, inplace = True)
#X_test.drop('customer_id', axis = 1, inplace = True)

X_test['released_year'] = X_test['released_year'].fillna(-999)
X_test['language'] = X_test['language'].fillna('none')
X_test['number_of_comments'] = X_test['number_of_comments'].fillna(-999)
print(X_test)

In [ ]:
#rough 8 --clf (scikit tree) predictor for test
X_test['customer_id']=X_test['customer_id'].astype('category')
X_test['language']=X_test['language'].astype('category')
print(X_test['language'])
cat_columns = X_test.select_dtypes(['category']).columns
print(cat_columns)
X_test[cat_columns] = X_test[cat_columns].apply(lambda x: x.cat.codes)
print(X_test['language'])
print(X_test.columns)
print(X_test['num_ratings'].isnull().values.any())
X_test['score_y']=X_test['score_y'].fillna(0)
X_test['num_ratings']=X_test['num_ratings'].fillna(0)
for i in range(max_labels):
    X_test[f'count{i}'] = X_test[f'count{i}'].fillna(0)

print(clf.predict(X_test))
print(Y_test)
loss = ((Y_test-clf.predict(X_test))**2).mean()

In [ ]:
#y_pred for multiple models (for multiple groups) case
y_pred=np.zeros(len(X_test))
for i in range(max_clusters):
    print('i: ',i,'/',max_clusters)
    X_testee=X_test[X_test.customer_id.isin(groups[i])]
    y_pred[X_testee.index]=models[i].predict(X_testee)
print('output length: ',len(y_pred))
print(y_pred)
#print(X_test.iloc[0])

In [ ]:
#rough 9

test_customers=X_test['customer_id']
y_test_pred=[]
for c in range(len(test_customers)):
    customer=test_customers[c]
    print(c,'/',len(test_customers),':',customer)
    ii=-1
    for i in range(len(groups.keys())):
        if customer in groups[i]:
            ii=i
            break
    if ii==0:
        y_test_pred.append(model0.predict(X_test.iloc[c]))
    elif ii==1:
        y_test_pred.append(model1.predict(X_test.iloc[c]))
    elif ii==2:
        y_test_pred.append(model2.predict(X_test.iloc[c]))
    elif ii==3:
        y_test_pred.append(model3.predict(X_test.iloc[c]))
#    elif ii==4:
#        y_test_pred.append(model4.predict(X_test.iloc[c]))
    else:
        print('issue!!')
        break

#y_test_pred = model.predict(X_test)M

In [ ]:
#writing data to a file!
#df=pd.DataFrame.from_dict(compressed_features,orient='index').transpose()
#df.to_csv('data/names.csv')
df=pd.DataFrame()
#df['test_row_id']=np.arange(len(y_test_pred))
df['score']=y_pred
print(df.head())
#x=pd.read_csv('./data/dummy_submission.csv')
df.to_csv('data/ff.csv')